In [1]:
import os
os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

/Users/nscherf/miniconda3/envs/dgl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Build an example of semi-supervised node classification. 

In [2]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

/Users/nscherf/.dgl/cora_v2.zip: 100%|██████████| 132k/132k [00:00<00:00, 44.2MB/s]


Extracting file to /Users/nscherf/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [3]:
len(dataset)

1

In [4]:
g = dataset[0]

In [6]:
g.ndata.keys()

dict_keys(['train_mask', 'val_mask', 'test_mask', 'label', 'feat'])

In [7]:
print("node features")
print(g.ndata['feat'])
print("node labels")
print(g.ndata['label'])

node features
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
node labels
tensor([3, 4, 4,  ..., 3, 3, 3])


In [9]:
print(g.edata)

{}


In [10]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [11]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        logits = model(g, features)
        pred = logits.argmax(1)
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(f"Epoch: {e}, Loss: {loss}, Train accuracy: {train_acc}, Val accuracy: {val_acc}, Test accuracy: {test_acc}")

model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

Epoch: 0, Loss: 1.9459667205810547, Train accuracy: 0.15714286267757416, Val accuracy: 0.14800000190734863, Test accuracy: 0.14399999380111694
Epoch: 5, Loss: 1.8930792808532715, Train accuracy: 0.6928571462631226, Val accuracy: 0.5519999861717224, Test accuracy: 0.5759999752044678
Epoch: 10, Loss: 1.8115625381469727, Train accuracy: 0.8642857074737549, Val accuracy: 0.6179999709129333, Test accuracy: 0.6399999856948853
Epoch: 15, Loss: 1.7063400745391846, Train accuracy: 0.8785714507102966, Val accuracy: 0.6119999885559082, Test accuracy: 0.6320000290870667
Epoch: 20, Loss: 1.5788606405258179, Train accuracy: 0.8857142925262451, Val accuracy: 0.6579999923706055, Test accuracy: 0.6800000071525574
Epoch: 25, Loss: 1.4329876899719238, Train accuracy: 0.9214285612106323, Val accuracy: 0.6880000233650208, Test accuracy: 0.7080000042915344
Epoch: 30, Loss: 1.2726044654846191, Train accuracy: 0.949999988079071, Val accuracy: 0.6980000138282776, Test accuracy: 0.7260000109672546
Epoch: 35, Lo

In [15]:
g.ndata['train_mask']

tensor([ True,  True,  True,  ..., False, False, False])